In [43]:
!pip install langchain langchain_community beautifulsoup4 chromadb sentence-transformers
!pip install -U lxml

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [82]:
from typing import Any, Dict, List, Optional, Sequence
from pathlib import Path
import re, html
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from IPython.display import Markdown, display, HTML

In [94]:
# --- injeta CSS uma única vez ---
_BLACK_CSS_INJECTED = False
# padrões (bloco e inline)
_RE_FENCE_TRIPLE_BACKTICKS = re.compile(r"```(.*?)```", flags=re.DOTALL)
_RE_FENCE_TRIPLE_QUOTES    = re.compile(r"'''(.*?)'''", flags=re.DOTALL)
_RE_INLINE_BACKTICK        = re.compile(r"`([^`]+)`")
# aspas simples com cuidado p/ não pegar apóstrofos em palavras
_RE_INLINE_SINGLE_QUOTE    = re.compile(r"(?<!\\w)'([^'\\n]+)'(?!\\w)")


In [56]:
def build_hf_embeddings(
    model_name: str = "intfloat/multilingual-e5-small",
    device: str = "cpu",
    normalize_embeddings: bool = True,
    **kwargs: Any,
) -> HuggingFaceEmbeddings:
    """
    Cria um objeto HuggingFaceEmbeddings compatível com o índice salvo.

    Parâmetros
    ----------
    model_name : str, opcional
        Nome do modelo de embeddings (ex.: "intfloat/multilingual-e5-small").
    device : str, opcional
        Dispositivo: "cpu" ou "cuda".
    normalize_embeddings : bool, opcional
        Se True, normaliza os vetores (útil para similaridade de cosseno).
    **kwargs : Any
        Pass-through para HuggingFaceEmbeddings (ex.: model_kwargs, encode_kwargs).

    Retorno
    -------
    HuggingFaceEmbeddings
        Instância configurada de embeddings.
    """
    model_kwargs = kwargs.pop("model_kwargs", {"device": device})
    encode_kwargs = kwargs.pop("encode_kwargs", {"normalize_embeddings": normalize_embeddings})
    print(f" Carregando embeddings '{model_name}' em '{device}' (normalize={normalize_embeddings})")
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        **kwargs,
    )


In [57]:
def load_chroma_index(
    embeddings: HuggingFaceEmbeddings,
    persist_directory: str,
) -> Chroma:
    """
    Carrega um índice vetorial Chroma previamente persistido em disco e informa contagem.

    Parâmetros
    ----------
    embeddings : HuggingFaceEmbeddings
        Objeto de embeddings *compatível* com o usado na criação do índice.
    persist_directory : str
        Caminho do diretório onde o índice foi salvo.

    Retorno
    -------
    Chroma
        Instância do vetor store pronta para uso (busca/RAG).
    """
    p = Path(persist_directory)
    if not p.exists():
        raise FileNotFoundError(
            f"O diretório '{persist_directory}' não existe. Crie o índice primeiro (notebook de guardar)."
        )

    print(f" 3. Carregando índice Chroma de '{persist_directory}' ...")
    vs = Chroma(
        embedding_function=embeddings,
        persist_directory=persist_directory,
    )
    # Contagem de itens (API interna do cliente do Chroma)
    try:
        n_items = vs._collection.count()  # type: ignore[attr-defined]
        print(f"   Coleção: '{vs._collection.name}' | Itens: {n_items}")  # type: ignore[attr-defined]
    except Exception:
        print("   (Não foi possível obter a contagem via _collection; prosseguindo mesmo assim)")

    return vs


In [58]:
def build_retriever(
    vectorstore: Chroma,
    k: int = 4,
    score_threshold: Optional[float] = None,
) -> Any:
    """
    Constrói um retriever para busca por similaridade no índice carregado.

    Parâmetros
    ----------
    vectorstore : Chroma
        Banco vetorial Chroma carregado.
    k : int, opcional
        Número de chunks a recuperar por consulta.
    score_threshold : float, opcional
        Limiar mínimo de score (quando suportado).

    Retorno
    -------
    BaseRetriever
        Retriever compatível com LangChain, pronto para compor a cadeia de RAG.
    """
    if score_threshold is None:
        print(f" 4. Criando retriever (k={k}) ...")
        return vectorstore.as_retriever(search_kwargs={"k": k})

    print(f" 4. Criando retriever (k={k}, score_threshold={score_threshold}) ...")
    return vectorstore.as_retriever(
        search_type="similarity_score_threshold",
        search_kwargs={"k": k, "score_threshold": score_threshold},
    )


In [59]:
def default_brazilian_prompt() -> PromptTemplate:
    """
    Prompt em PT-BR para respostas didáticas com uso de contexto.

    Retorno
    -------
    PromptTemplate
        Template com variáveis: "context" e "question".
    """
    template = (
        "Use o contexto a seguir para responder à pergunta no final.\n"
        "Se você não souber, diga que não sabe — não invente.\n"
        "Responda em Português do Brasil, com clareza e exemplos passo a passo quando fizer sentido.\n\n"
        "{context}\n\n"
        "Pergunta: {question}\n"
        "Resposta útil:"
    )
    return PromptTemplate(template=template, input_variables=["context", "question"])


In [60]:
def build_ollama_llm(
    model: str = "llama3.2:3b-instruct-q4_K_M",
    temperature: float = 0.0,
    **kwargs: Any,
) -> Ollama:
    """
    Prepara o LLM do Ollama para uso no RAG.

    Parâmetros
    ----------
    model : str, opcional
        Nome do modelo disponível no Ollama (`ollama list`).
    temperature : float, opcional
        Temperatura de amostragem.
    **kwargs : Any
        Parâmetros extras para `Ollama` (ex.: base_url, num_ctx).

    Retorno
    -------
    Ollama
        Instância do LLM configurada.
    """
    print(f" 5. Preparando LLM Ollama (model={model}, temperature={temperature}) ...")
    return Ollama(model=model, temperature=temperature, **kwargs)


In [61]:
def build_retrieval_qa_chain(
    llm: Ollama,
    retriever: Any,
    prompt: PromptTemplate,
    chain_type: str = "stuff",
    return_sources: bool = True,
) -> RetrievalQA:
    """
    Monta a cadeia RAG (Retriever + Prompt + LLM) para responder perguntas com contexto.

    Parâmetros
    ----------
    llm : Ollama
        Modelo de linguagem preparado.
    retriever : BaseRetriever
        Mecanismo de recuperação a partir do Chroma.
    prompt : PromptTemplate
        Template com variáveis "context" e "question".
    chain_type : str, opcional
        Tipo da cadeia ("stuff", "map_reduce", "refine").
    return_sources : bool, opcional
        Se True, retorna documentos-fonte.

    Retorno
    -------
    RetrievalQA
        Cadeia pronta para executar perguntas.
    """
    print(" 6. Montando cadeia de RAG ...")
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type=chain_type,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=return_sources,
    )


In [92]:
def format_sources(source_documents: Optional[Sequence[Any]]) -> List[str]:
    """
    Converte Documents de fonte em strings amigáveis (caminho + score quando disponível).

    Parâmetros
    ----------
    source_documents : Sequence[Document] | None
        Documentos retornados pelo retriever/chain.

    Retorno
    -------
    List[str]
        Lista de descrições de fonte.
    """
    items: List[str] = []
    for doc in source_documents or []:
        meta = getattr(doc, "metadata", {}) or {}
        origem = meta.get("source") or meta.get("file_path") or str(meta)
        score = meta.get("score")
        if score is not None:
            origem = f"{origem} (score={score})"
        items.append(origem)
    return items


In [93]:
def ensure_black_code_css() -> None:
    """Injeta CSS para caixas de código com fundo preto e texto branco (inline e bloco)."""
    global _BLACK_CSS_INJECTED
    if _BLACK_CSS_INJECTED:
        return
    css = """
    <style>
      /* Inline "pílula" */
      .codechip {
        display: inline-block !important;
        background: #000 !important;
        color: #fff !important;
        padding: 0 6px !important;
        border-radius: 6px !important;
        line-height: 1.4 !important;
        vertical-align: baseline !important;
        font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, "Liberation Mono", monospace !important;
      }
      .codechip code {
        background: transparent !important;
        color: inherit !important;
        padding: 0 !important;
      }

      /* Bloco */
      .codeblock {
        display: block !important;
        background: #000 !important;
        color: #fff !important;
        padding: 10px 12px !important;
        border-radius: 8px !important;
        white-space: pre-wrap !important;
        overflow: auto !important;
        font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, "Liberation Mono", monospace !important;
        margin: .6em 0 !important;
      }
      .codeblock code {
        background: transparent !important;
        color: inherit !important;
        padding: 0 !important;
      }
    </style>
    """
    display(HTML(css))
    _BLACK_CSS_INJECTED = True

In [95]:
def _wrap_block(code: str) -> str:
    """Envolve como bloco (caixa preta maior)."""
    return f"<pre class='codeblock'><code>{html.escape(code.strip())}</code></pre>"

In [96]:
def _wrap_inline(code: str) -> str:
    """Envolve como pílula inline (apenas um pouco maior que o conteúdo)."""
    return f"<span class='codechip'><code>{html.escape(code.strip())}</code></span>"

In [97]:
def format_code_blocks_black(text: str) -> str:
    """
    Converte:
      - ```bloco```,  '''bloco'''  → bloco com fundo preto
      - `inline`, 'inline'          → pílula inline com fundo preto
    Garante contraste (texto branco) e evita CSS do tema sobrescrever.
    """
    placeholders: List[str] = []

    def sub_block(pattern: re.Pattern, s: str) -> str:
        def push(m: re.Match) -> str:
            placeholders.append(_wrap_block(m.group(1)))
            return f"@@CODE{len(placeholders)-1}@@"
        return pattern.sub(push, s)

    def sub_inline(pattern: re.Pattern, s: str) -> str:
        def push(m: re.Match) -> str:
            placeholders.append(_wrap_inline(m.group(1)))
            return f"@@CODE{len(placeholders)-1}@@"
        return pattern.sub(push, s)

    # 1) blocos primeiro
    tmp = sub_block(_RE_FENCE_TRIPLE_BACKTICKS, text)
    tmp = sub_block(_RE_FENCE_TRIPLE_QUOTES,    tmp)
    # 2) inlines depois
    tmp = sub_inline(_RE_INLINE_BACKTICK,       tmp)
    tmp = sub_inline(_RE_INLINE_SINGLE_QUOTE,   tmp)

    # 3) escapa tudo que sobrou (texto comum), para não virar HTML acidental
    safe = html.escape(tmp)

    # 4) recoloca os HTML reais
    for i, html_block in enumerate(placeholders):
        token = f"@@CODE{i}@@"
        safe = safe.replace(html.escape(token), html_block).replace(token, html_block)

    return safe

In [98]:
def ask(
    chain: RetrievalQA,
    question: str,
    show_sources: bool = True,
) -> Dict[str, Any]:
    """
    Executa uma pergunta na cadeia RAG e retorna resposta + fontes.
    Trechos entre:
      - ```bloco```,  '''bloco'''
      - `inline`,     'inline'
    saem com **fundo preto** e **texto branco** (inline = “pílula”, bloco = caixa).
    """
    print(f" 7. Pergunta: {question}")
    resp = chain(question)

    # injeta CSS (uma vez) e prepara HTML
    ensure_black_code_css()
    answer_html = format_code_blocks_black(resp.get("result", ""))

    print("\n Resposta:\n")
    display(HTML(answer_html))

    # fontes
    srcs: List[str] = []
    for doc in resp.get("source_documents", []) or []:
        meta = getattr(doc, "metadata", {}) or {}
        origem = meta.get("source") or meta.get("file_path") or str(meta)
        score = meta.get("score")
        if score is not None:
            origem = f"{origem} (score={score})"
        srcs.append(origem)

    if show_sources:
        print("\n---\n Fontes:")
        for i, s in enumerate(srcs, 1):
            print(f"[{i}] {s}")

    return {"answer": answer_html, "sources": srcs, "raw": resp}

In [103]:
# === Parâmetros ===
PERSIST_DIR = "./chroma_db_python_iniciante"      # mesmo diretório usado no notebook de guardar
EMBEDDINGS_MODEL = "intfloat/multilingual-e5-small"
DEVICE = "cpu"                                     # ou "cuda"
NORMALIZE = True
K = 4                                              # número de chunks retornados
OLLAMA_MODEL = "llama3.2:3b-instruct-q4_K_M"
TEMPERATURE = 0.0

# === Pipeline ===
emb = build_hf_embeddings(
    model_name=EMBEDDINGS_MODEL,
    device=DEVICE,
    normalize_embeddings=NORMALIZE
)

vs = load_chroma_index(
    embeddings=emb,
    persist_directory=PERSIST_DIR
)

retriever = build_retriever(
    vectorstore=vs,
    k=K,
    score_threshold=None  # ou por exemplo 0.2, se quiser filtrar
)

prompt = default_brazilian_prompt()
llm = build_ollama_llm(model=OLLAMA_MODEL, temperature=TEMPERATURE)
chain = build_retrieval_qa_chain(llm, retriever, prompt)

# === Perguntar ===
QUESTION = "defina o que é um método e de exemplos"
result = ask(chain, QUESTION, show_sources=True)


 Carregando embeddings 'intfloat/multilingual-e5-small' em 'cpu' (normalize=True)
 3. Carregando índice Chroma de './chroma_db_python_iniciante' ...
   ➜ Coleção: 'langchain' | Itens: 9293
 4. Criando retriever (k=4) ...
 5. Preparando LLM Ollama (model=llama3.2:3b-instruct-q4_K_M, temperature=0.0) ...
 6. Montando cadeia de RAG ...
 7. Pergunta: defina o que é um método e de exemplos

 Resposta:




---
 Fontes:
[1] data/python-3.13-docs-html/reference/datamodel.html
[2] data/python-3.13-docs-html/glossary.html
[3] data/python-3.13-docs-html/reference/datamodel.html
[4] data/python-3.13-docs-html/glossary.html
